## Introdução

Aqui irei realizar a etapa de Feature Engineering na base de dados `bureau_balance`.

Esta etapa está dividida em kernels diferentes devido o consumo de memória para carregar e manipular as bases.

Os procedimentos realizados nas outras bases podem ser encontrados nos arquivos `02_feat_eng_<nome-da-base>.ipynb`

O objetivo desta etapa consiste, principalmente, em criar variáveis (`book de variáveis`). Ao criar novas variáveis com base nas variáveis existentes, é possível capturar informações adicionais que podem não estar explicitamente presentes nos dados originais.

Devido o volume de dados, optei por utilizar o PySpark em conjunto com o SparkSQL para as operações a seguir.

* Sobre os Dados

A base `bureau balance` possui informações mensais sobre créditos anteriores do cliente em outras instituições financeiras.

Segundo os Metadados disponibilizados, essas são as informações contidas aqui:

``SK_BUREAU_ID``: ID recodificado do crédito do Bureau de Crédito (codificação única para cada aplicação) - Será usado para fazer a junção com a tabela `BUREAU`.


``MONTHS_BALANCE``: Mês do saldo em relação à data de aplicação (-1 significa a data de saldo mais recente).


``STATUS``: Status do empréstimo do Bureau de Crédito durante o mês (ativo, fechado, DPD0-30,... [C significa fechado, X significa status desconhecido, 0 significa sem DPD, 1 significa atraso máximo durante o mês entre 1-30, 2 significa DPD 31-60,... 5 significa DPD 120+ ou vendido ou baixado]).

## Utils

* Importando as bibliotecas que irei utilizar

In [1]:
import os
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import when,min, max, sum, round, col, median, count
spark = SparkSession.builder \
    .appName("FeatureEng") \
    .config("spark.executor.memory", "14g") \
    .config("spark.driver.memory", "14g") \
    .getOrCreate()
from warnings import filterwarnings
filterwarnings('ignore')

## Feature Engineering - Bureau Balance

In [2]:
bureau_balance = spark.read.csv('./DATASETS/bureau_balance.csv', inferSchema= True, header= True)
bureau_balance.count()

27299925

In [3]:
bureau_balance.show(n=10)

+------------+--------------+------+
|SK_ID_BUREAU|MONTHS_BALANCE|STATUS|
+------------+--------------+------+
|     5715448|             0|     C|
|     5715448|            -1|     C|
|     5715448|            -2|     C|
|     5715448|            -3|     C|
|     5715448|            -4|     C|
|     5715448|            -5|     C|
|     5715448|            -6|     C|
|     5715448|            -7|     C|
|     5715448|            -8|     C|
|     5715448|            -9|     0|
+------------+--------------+------+
only showing top 10 rows



* Checando a quantidade de linhas da tabela final (para validação posterior)

In [4]:
bureau_balance.groupBy("SK_ID_BUREAU").count().count()

817395

### 2. Criação de Flags Temporais

In [5]:
# Criando uma View
bureau_balance.createOrReplaceTempView('bureau_balance')

temp01 = spark.sql("""
SELECT
    *,
        CASE
            WHEN MONTHS_BALANCE >= -3 THEN 1
        ELSE 0
    END AS FL_U3M,
        CASE
            WHEN MONTHS_BALANCE >= -6 THEN 1
        ELSE 0
    END AS FL_U6M,
        CASE
            WHEN MONTHS_BALANCE >= -9 THEN 1
        ELSE 0
    END AS FL_U9M,
        CASE
            WHEN MONTHS_BALANCE >= -12 THEN 1
        ELSE 0
    END AS FL_U12M
FROM
    bureau_balance
ORDER BY
    `SK_ID_BUREAU`;
""")

temp01.createOrReplaceTempView('temp01')
temp01.count()

27299925

In [6]:
temp01.show(n=5)

+------------+--------------+------+------+------+------+-------+
|SK_ID_BUREAU|MONTHS_BALANCE|STATUS|FL_U3M|FL_U6M|FL_U9M|FL_U12M|
+------------+--------------+------+------+------+------+-------+
|     5001709|             0|     C|     1|     1|     1|      1|
|     5001709|            -5|     C|     0|     1|     1|      1|
|     5001709|            -1|     C|     1|     1|     1|      1|
|     5001709|            -2|     C|     1|     1|     1|      1|
|     5001709|            -3|     C|     1|     1|     1|      1|
+------------+--------------+------+------+------+------+-------+
only showing top 5 rows



### 3. Criação das Flags das Variáveis Categóricas

In [7]:
# Lista para armazenar as colunas criadas
flags = []

# Pegando todas as colunas categoricas da tabela
cat_cols = [cat_col[0] for cat_col in temp01.dtypes if cat_col[1] == 'string']

# Gerando as colunas Flag
for cat_col in cat_cols:
    unique_vals = [col[0] for col in temp01.select(cat_col).distinct().collect()]

    for unique_val in unique_vals:
        flags.append(when(col(cat_col) == unique_val, 1).otherwise(0).alias(f'FL_CAT_{cat_col}_{unique_val.upper().replace(" ", "_")}'))

* Realizando o Unpacking e Criando a Tabela

In [8]:
temp02 = temp01.select("*", *flags)

In [9]:
temp02.show(n=5)

+------------+--------------+------+------+------+------+-------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|SK_ID_BUREAU|MONTHS_BALANCE|STATUS|FL_U3M|FL_U6M|FL_U9M|FL_U12M|FL_CAT_STATUS_3|FL_CAT_STATUS_0|FL_CAT_STATUS_5|FL_CAT_STATUS_C|FL_CAT_STATUS_X|FL_CAT_STATUS_1|FL_CAT_STATUS_4|FL_CAT_STATUS_2|
+------------+--------------+------+------+------+------+-------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|     5001709|             0|     C|     1|     1|     1|      1|              0|              0|              0|              1|              0|              0|              0|              0|
|     5001709|            -5|     C|     0|     1|     1|      1|              0|              0|              0|              1|              0|              0|              0|              0|
|     5001709|            -1| 

### 4. Criação das Variáveis (Agrupadas)

* Filtrando as colunas que serão agregadas

In [10]:
flags_temporais = ['FL_U3M', 'FL_U6M', 'FL_U9M','FL_U12M']

flags_categoricas = [col for col in temp02.columns if 'FL_CAT_' in col]

In [11]:
new_cols = []


for temp_flag in flags_temporais:
    nome_temp_flag = temp_flag.replace("FL_","").upper()
    for cat_flag in flags_categoricas:
        nome_cat_flag = cat_flag.replace("FL_CAT_","").upper()
        new_cols.append(round(count(when(col(temp_flag) == 1, col(cat_flag))),2).alias(f"QT_{nome_cat_flag}_{nome_temp_flag}"))

new_cols = tuple(new_cols)

temp03 = temp02.groupBy("SK_ID_BUREAU").agg(*new_cols).orderBy("SK_ID_BUREAU")

temp03.count()

817395

In [12]:
print("Quantidade de Vars. Criadas:",len(temp03.columns))

Quantidade de Vars. Criadas: 33


In [13]:
temp03.show(n=10)

+------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+
|SK_ID_BUREAU|QT_STATUS_3_U3M|QT_STATUS_0_U3M|QT_STATUS_5_U3M|QT_STATUS_C_U3M|QT_STATUS_X_U3M|QT_STATUS_1_U3M|QT_STATUS_4_U3M|QT_STATUS_2_U3M|QT_STATUS_3_U6M|QT_STATUS_0_U6M|QT_STATUS_5_U6M|QT_STATUS_C_U6M|QT_STATUS_X_U6M|QT_STATUS_1_U6M|QT_STATUS_4_U6M|QT_STATUS_2_U6M|QT_STATUS_3_U9M|QT_STATUS_0_U9M|QT_STATUS_5_U9M|QT_STATUS_C_U9M|QT_STATUS_X_U9M|QT_STATUS_1_U9M|QT_STATUS_4_U9M|QT_STATUS_2_U9M|QT_STATUS_3_U12M|QT_STATUS_0_U12M|QT_STATUS_5_U12M|QT_STATUS_C_U12M

### 5. Salvando a Tabela Final

In [14]:
bureau_balance_agg = temp03.repartition(1)

bureau_balance_agg.write.mode('overwrite').option("compression", "gzip").parquet("./BASES_FEAT_ENG/BUREAU_BALANCE_FEAT_ENG")